In [2]:
import wfdb
from wfdb.plot import plot
import matplotlib.pyplot as plt
import cv2
from glob import glob
import numpy as np
directory="D:/ProyectoECG\mit-bih"
print("Working")

# record = wfdb.rdrecord('/Users/wilbe/OneDrive/Público/Escritorio/ProyectoECG/mit-bih/101', sampto=3000)
# ann = wfdb.rdann('/Users/wilbe/OneDrive/Público/Escritorio/ProyectoECG/mit-bih/101', 'atr', sampto=3000)
# wfdb.plot_wfdb(record=record, annotation=ann, plot_sym=True,
#                     time_units='seconds', title='MIT-BIH Record 100',
#                     figsize=(10,4), ecg_grids='all')

num_images=100

Working


In [3]:
def get_records():
    """ Obtener los paths para los archivos"""
    
    # Son 3 tipos de archivo
    paths = glob(directory+"/*.atr")

    #quita la extensión
    paths = [path[:-4] for path in paths]
    paths.sort()
    #print("I set paths")
    
    paths.remove('D:/ProyectoECG\mit-bih\\102-0')
    #print(paths)
    return paths

def beat_annotations(annotation, tipo):
    
    
    good = [tipo]   
    ids = np.in1d(annotation.symbol, good)

    # Posiciones de lo que contengan good
    beats = annotation.sample[ids]

    return beats
  
def segmentation(records,tipo):
    Normal = []
    for e in records:
        signals, fields = wfdb.rdsamp(e, channels = [0]) 
        #print("Segmentando")
        ann = wfdb.rdann(e, 'atr')
        good = [tipo]
        ids = np.in1d(ann.symbol, good)
        imp_beats = ann.sample[ids]
        beats = (ann.sample)
        for i in imp_beats:
            beats = list(beats)
            j = beats.index(i)
            if(j!=0 and j!=(len(beats)-1)):
                x = beats[j-1]
                y = beats[j+1]
                diff1 = abs(x - beats[j])//2
                diff2 = abs(y - beats[j])//2
                Normal.append(signals[beats[j] - diff1: beats[j] + diff2, 0])
    return Normal

In [4]:
def get_images(segmentos,inicial,path_):
    #print("Worked?")
    acum=0
    for count, i in enumerate(segmentos):
        #print("Dentro")
        plt.ioff()
        fig = plt.figure(frameon=False)
        plt.plot(i) 
        plt.xticks([]), plt.yticks([])
        for spine in plt.gca().spines.values():
            spine.set_visible(False)

        # filename = directory + '/mit-imagenes/' + str(count+inicial)+'.png'
        # fig.savefig(filename)
        # filename = './mit-bih/mit-imagenes/' + str(count+inicial)+'.png'

        filename = directory + path_ + str(count+inicial)+'.png'
        fig.savefig(filename)
        filename = './mit-bih' +path_+ str(count+inicial)+'.png'
        
        #print("Este es el path de la img",filename)
        im_gray = cv2.imread(filename,cv2.IMREAD_GRAYSCALE)
        #print("Estes es el shape de la img",im_gray.shape)
        im_gray = cv2.resize(im_gray, (128, 128), interpolation = cv2.INTER_LANCZOS4)
        
        cv2.imwrite(filename, im_gray)
        plt.close()
        #print("Fin")
        acum+=1
        if acum%num_images==0: #Only testing with 5 of each class
            break
    return inicial+acum

In [5]:
#Leyendo imagen

import cv2


#Le das la imagen, el filename="100.png", path_="/mit-imagenes/PAV/"
def cropping(image, filename,path_):
    #path_=path_+"/"
    #path_ = './mit-bih' +path_+ 'cropping/' +filename[:-4]
    """path_= ./mit-bih/mit-imagenes/PAV/cropping/ """
    path_=path_+filename[:-4]
    ####################
    #TOP
    #Izquierda
    crop = image[:96, :96]
    crop = cv2.resize(crop, (128, 128))
    cv2.imwrite(path_ + 'leftTop' + '.png', crop)
    
    #Centro
    crop = image[:96, 16:112]
    crop = cv2.resize(crop, (128, 128))
    cv2.imwrite(path_ + 'centerTop' + '.png', crop)
    
    #Derecha
    crop = image[:96, 32:]
    crop = cv2.resize(crop, (128, 128))
    cv2.imwrite(path_ + 'rightTop' + '.png', crop)
    
    ####################
    #Centro
    #Izquierda
    crop = image[16:112, :96]
    crop = cv2.resize(crop, (128, 128))
    cv2.imwrite(path_ + 'leftCenter' + '.png', crop)
    
    #Centro
    crop = image[16:112, 16:112]
    crop = cv2.resize(crop, (128, 128))
    cv2.imwrite(path_ + 'centerCenter' + '.png', crop)
    
    #Derecha
    crop = image[16:112, 32:]
    crop = cv2.resize(crop, (128, 128))
    cv2.imwrite(path_ + 'rightCenter' + '.png', crop)
    

    ####################
    #Inferior
    #Izquierda

    crop = image[32:, :96]
    crop = cv2.resize(crop, (128, 128))
    cv2.imwrite(path_ + 'leftBottom' + '.png', crop)
    
    #Centro
    crop = image[32:, 16:112]
    crop = cv2.resize(crop, (128, 128))
    cv2.imwrite(path_ + 'centerBottom' + '.png', crop)
    
    #Derecha
    crop = image[32:, 32:]
    crop = cv2.resize(crop, (128, 128))
    cv2.imwrite(path_ + 'rightBottom' + '.png', crop)

CREAR IMÁGENES DE TODOS LOS TIPOS A EVALUAR


In [6]:

############################################
rec_=get_records()          #Obtener los paths de los records
#beats_ann=beat_annotations()

"""
PVC=V       premature ventricular contraction beat
PAB=/       paced beat
RBB=R       right bundle branch block beat
LBB=L       left bundle branch block beat
APC=A       atrial premature contraction beat
VFW=!       ventricular flutter wave
VEB=E       ventricular escape beat
N=N         normal

https://physionet.org/physiobank/database/html/mitdbdir/intro.htm#symbols"""
def returnPath(ch):
    path="/mit-imagenes/"
    agregar=""
    if ch == 'V':
        agregar="PVC"
    elif ch == '/':
        agregar="PAB"
    elif ch == 'R':
        agregar="RBB"
    elif ch == 'L':
        agregar="LBB"
    elif ch == 'A':
        agregar="APC"
    elif ch == '!':
        agregar="VFW"
    elif ch == 'E':
        agregar="VEB"
    elif ch == 'N':
        agregar="N"
    path=path+agregar+'/'
    """se retornaría el path="/mit-imagenes/PVC/" """
    return path
#tipos=['N','L','R','A','V','/','E']

tipos=['V','/','R','L','A','!','E','N']
#tipos=['N']

# tipoPath=returnPath(tipos[1])

# contador=0
# segmentos=segmentation(rec_,tipos[0])
# contador=get_images(segmentos,contador,tipoPath)
# cropping()


for i in tipos:
   tipoPath=returnPath(i)
   segmentos=segmentation(rec_,i)
   contador=get_images(segmentos,contador,tipoPath)
   
#segmentos=segmentation(rec_)

# obj=get_images(segmentos)


RECORTANDO IMÁGENES

In [12]:
import os
#Obteniendo las imágenes de un folder path_="./mit-bih/mit-imagenes/PAB/"
def cropping_images_folder(path_):
    images = []
    path_='./mit-bih'+path_
    final_path_=path_+'cropping/'     #
    
    #print(path_)
    print(final_path_)
    count=1
    for filename in os.listdir(path_):
        if count%100==0:        #Only cropping 100 images
            break
        #print(os.path.join(path_,filename))
        #print("The filename is",filename)
        img = cv2.imread(os.path.join(path_,filename))
        if img is not None:
            #print("is image")
        #     images.append(img)
            cropping(img,filename,final_path_)
            count+=1
        print("Counting cropped images: ",count)
    return images


tipos_crop=['V','/','R','L','A','!','E']
#Recortando las imágenes de todos los folders excepto N
for i in tipos_crop:
    print("Cropping Folder: ", i)
    tipoPath=returnPath(i)
    print("path of folder: ",tipoPath)
    cropping_images_folder(tipoPath)


""" PROBANDO Cropping
tipoPath=returnPath(tipos[1])
#pass /mit-imagenes/PAB/
#print(tipoPath)
cropping_images_folder(tipoPath)"""


Cropping Folder:  V
path of folder:  /mit-imagenes/PVC/
./mit-bih/mit-imagenes/PVC/cropping/
Counting cropped images:  2
Counting cropped images:  3
Counting cropped images:  4
Counting cropped images:  5
Counting cropped images:  6
Counting cropped images:  7
Counting cropped images:  8
Counting cropped images:  9
Counting cropped images:  10
Counting cropped images:  11
Counting cropped images:  12
Counting cropped images:  13
Counting cropped images:  14
Counting cropped images:  15
Counting cropped images:  16
Counting cropped images:  17
Counting cropped images:  18
Counting cropped images:  19
Counting cropped images:  20
Counting cropped images:  21
Counting cropped images:  22
Counting cropped images:  23
Counting cropped images:  24
Counting cropped images:  25
Counting cropped images:  26
Counting cropped images:  27
Counting cropped images:  28
Counting cropped images:  29
Counting cropped images:  30
Counting cropped images:  31
Counting cropped images:  32
Counting cropped

' PROBANDO Cropping\ntipoPath=returnPath(tipos[1])\n#pass /mit-imagenes/PAB/\n#print(tipoPath)\ncropping_images_folder(tipoPath)'